In [ ]:
import sqlite3
import json as j
import pandas as pd
import numpy as np
from potato import *

In [ ]:
dbpath="/home/sergei/.qif-transaction-generator/db.sqlite"
conn = sqlite3.connect(dbpath)
c = conn.cursor()

In [ ]:
c.execute("""
SELECT
	r.raw,
	r.id,
	r.total,
	i.id,
	i.name,
	i.price,
	i.quantity,
	i.sum,
	a.guid,
	a.name,
	a.parent_guid,
	a.full_name
FROM
	receipts r 
	join items i 
	on r.id = i.receipt_id
	join accounts a 
	on i.account_guid = a.guid 
WHERE
	purchase_date > '2022-01-01'
	and raw like '%\"user\":\"ООО \\\"Лента\\\"%'""")
data = c.fetchall()

In [ ]:
df = pd.DataFrame(data, columns = ['raw',
'receipt_id',
'total',
'item_id',
'item_name',
'price',
'quantity',
'sum',
'guid',
'account_name',
'parent_guid',
'full_name'
])
df['date'] = pd.to_datetime(df['raw'].apply(lambda x: j.loads(x)['localDateTime']))
df['month'] = df['date'].dt.to_period('M')
df['total'] = df['total']/100
df['sum'] = df['sum']/100
df['price'] = df['price']/100

In [ ]:
lenta_accounts = pd.DataFrame({'guid': ['88c27f35985a456e93c892f9e103aaa5', 
'bc800979983f25b1618b0838d7ade3e6', 
'5410b7a9074d04358bdaa07fd348bc68', 
'2935651d4e0df737c27a03dda98d3820', 
'24b40db51a6e3384cc0cf9c67977d5c2', 
'16bd1484e2140947c4de0eb3dfd0275c', 
'c086e0849cc098cb0ac771b19d36b912', 
'468ea6c80dff4691aa63761979dde84c', 
'087d9df31312eabbe555a87e38c07fd8', 
'f4c3607271d669f32b0b6ce9e37ac47d', 
'caca21a694c7cebd6aee930c27995989', 
'b9f44576ef880ccf7dd7533f0da11f50', 
'e21919cab28b84e2142e991129b1175f', 
'd6f8726e0603c165727e78c3e7336a6d', 
'10ee1db863fd803544e53799f5fb2912', 
'c75da7b28bf86c14c12af1cfc8a1eaa7', 
'33a394f97d03cee5c6dd7b8c9a7ecfa0', 
'4276c02639128bbdc14fc1c3acf175db', 
'd21ca453c2b73eaeb429cf14b09994bf', 
'7988312dfb881d3f515e135dcdc571fb', 
'21752f3b4519bee5074e5c6f4273c7ee', 
'90dba228f780742a47f633412d224231', 
'cdc7300c3dcf5592b571fad04933e1f0', 
'85923be17069c81b23f2a9641426f1eb', 
'42474fc596d1c29dc4a651486d44a6de', 
'859fcaff217309b31cda021c50ee691a', 
'0af97ed200a6eb350498e8a031b471ce', 
'2fc7be4f05ca06a69498f1768224678d', 
'81afc999e01cf559d89820a454c971b2', 
'd90a00af9493a063bdde2b9eb2c251f3', 
'e7fb50b10dbf9ee113597984f535f96f', 
'f9599035cac0aed46c1ecadeb3c520d1', 
'f7e0fec4497d624f3d02aa44afeb92d1', 
'9c01827b7e5a91e5cfadadf98cc41424', 
'7114ef07f047ba7e9faa48d31a221eec', 
'f97747b23b5c247b8c00fdaaffbf7021', 
'929ef56cbf83997a5aa6b96056c12a9a', 
'025519b95971c1cc6f1c11324e7f900e', 
'15a9220213ef6ac9cc7852bf5777872a'],
'lenta_category': ['Молочные продукты',
'Красота и гигиена',
'Вода',
'Сладости',
'Красота и гигиена',
'Красота и гигиена',
'Красота и гигиена',
'Детские товары',
'Молочные продукты',
'Молочные продукты',
'Консервы',
'Консервы',
'Крупы',
'Макароны',
'Молочные продукты',
'Молочные продукты',
'Сладости',
'Бытовая химия',
'Красота и гигиена',
'Овощи',
'Орехи',
'Орехи',
'Сладости',
'Сладости',
'Крупы',
'Рыба и морепродукты',
'Сладости',
'Молочные продукты',
'Бытовая химия',
'Сухофрукты',
'Сыры',
'Молочные продукты',
'Красота и гигиена',
'Мясо',
'Фрукты',
'Чай',
'Красота и гигиена',
'Сладости',
'Яйцо'
]})

In [ ]:
df = df.join(lenta_accounts.set_index('guid'), on='guid').sort_values(by='date')
#df['lenta_category']=df['lenta_category'].fillna('')
df.dropna(subset=['lenta_category'], inplace=True)

In [ ]:
cum_sum_by_receipts_df=df[['receipt_id', 'total', 'month', 'date']].drop_duplicates(['total', 'date'])
cum_sum_by_receipts_df['cum_total']=cum_sum_by_receipts_df.groupby(['month'])['total'].cumsum()
cum_sum_by_receipts_df['cum_total']=cum_sum_by_receipts_df['cum_total']-cum_sum_by_receipts_df['total']

In [ ]:
def print_df(df):
    with pd.option_context('display.max_rows', None,
                           'display.max_columns', None,
                           'display.precision', 3,
                           ):
        display(df)

In [ ]:
# Статистика трат:
display(cum_sum_by_receipts_df)
cum_sum_by_receipts_df.groupby(['month']).sum()

In [ ]:
df=df.join(cum_sum_by_receipts_df[['receipt_id', 'cum_total']].set_index('receipt_id'), on='receipt_id')
df['cum_total']=df['cum_total'].fillna(0)
df['sum']=df['sum'].fillna(0)

In [ ]:
categories = [
	Category('Молочные продукты', 1500),
	Category('Фрукты', 3000),
	Category('Овощи', 6000),
	Category('Сыры', 10000),
	Category('Рыба и морепродукты', 15000),
]
def calculate_bonus(row):
    active_categories=get_active_categories(categories, row['cum_total'])
    if row['lenta_category'] in active_categories:
        return round(0.05*row['sum'])
    else:
        return 0
df['bonus']=df.apply(lambda x: calculate_bonus(x) , axis=1)

In [ ]:
# display(df)

In [ ]:
#df.groupby(['month', 'lenta_category'])[['sum']].sum().reset_index().sort_values(by=['month', 'sum'])
result=df.groupby(['month', 'lenta_category'])[['sum']].sum().sort_values(['month', 'sum'], ascending=(True, False)).groupby('month').head(5)
result

In [ ]:
result.reset_index().plot.bar(rot=0, x='month', y=[])

In [ ]:
# import seaborn as sns
# sns.set_style("dark")
plot_data=pd.pivot_table(result, index=['month'], columns=['lenta_category'], fill_value=0)
display(plot_data)
plot_data.plot.bar(
    stacked=True,
    figsize=(20,10)
).legend(
    loc="upper left"
)